<a href="https://colab.research.google.com/github/binhdodata/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
uploaded = files.upload()


Saving charity_data.csv to charity_data.csv


1. Import Dependencies and Load Data:

In [10]:
# Import dependencies
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Load data
data_path = "charity_data.csv"
df = pd.read_csv(data_path)


Preprocess the Data:

In [11]:
# Drop non-beneficial columns
df = df.drop(columns=['EIN', 'NAME'])

# Binning the APPLICATION_TYPE column
application_counts = df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are below a certain threshold
threshold = 500
application_to_replace = list(application_counts[application_counts < threshold].index)

# Replace in dataframe
for app in application_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful
print(df['APPLICATION_TYPE'].value_counts())

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


Binning the CLASSIFICATION column:


In [12]:
classification_counts = df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are below a certain threshold
threshold = 1500  # You can adjust this threshold based on your understanding of the data
classification_to_replace = list(classification_counts[classification_counts < threshold].index)

# Replace in dataframe
for cls in classification_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure binning was successful
print(df['CLASSIFICATION'].value_counts())


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [16]:
# Encode categorical variables using one-hot encoding
df_encoded = pd.get_dummies(df)

# Split the dataset into features and target arrays
y = df_encoded["IS_SUCCESSFUL"].values
X = df_encoded.drop(["IS_SUCCESSFUL"], 1).values

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Scale the datasets
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

<ipython-input-16-4601256e2d30>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df_encoded.drop(["IS_SUCCESSFUL"], 1).values


Design Neural Network Model

In [17]:
# Initialize the model
nn_optimized = tf.keras.models.Sequential()

# Add input and hidden layers
nn_optimized.add(tf.keras.layers.Dense(units=100, activation='relu', input_dim=len(X_train_scaled[0])))
nn_optimized.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn_optimized.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Add output layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
nn_optimized.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Train and Evaluate the Model

In [18]:
# Train the model
fit_model = nn_optimized.fit(X_train_scaled, y_train, epochs=100)  # you can adjust epochs based on performance

# Evaluate the model's performance
model_loss, model_accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 9s 6ms/step - loss: 0.5690 - accuracy: 0.7227
Epoch 2/100
804/804 [==============================] - 6s 8ms/step - loss: 0.5547 - accuracy: 0.7315
Epoch 3/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5515 - accuracy: 0.7316
Epoch 4/100
804/804 [==============================] - 5s 5ms/step - loss: 0.5504 - accuracy: 0.7332
Epoch 5/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5491 - accuracy: 0.7335
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5486 - accuracy: 0.7338
Epoch 7/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5476 - accuracy: 0.7352
Epoch 8/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5469 - accuracy: 0.7351
Epoch 9/100
804/804 [==============================] - 6s 8ms/step - loss: 0.5469 - accuracy: 0.7352
Epoch 10/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5459 - accura

In [20]:
from tensorflow.keras.models import load_model

# Load the model
loaded_nn = load_model("AlphabetSoupCharity_Optimization.h5")

# Use the loaded model as you would the original
loaded_nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4400      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 30)                1530      
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 11011 (43.01 KB)
Trainable params: 11011 (43.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
